In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Imports and definitions

In [13]:
import numpy as np
import pandas as pd
from datetime import datetime
import xarray as xr
from pathlib import Path

In [14]:
from ecephys_analyses.data import channel_groups, paths
from ecephys.sglx_utils import load_timeseries
import ecephys.signal.timefrequency as tfr

In [15]:
xr.set_options(keep_attrs=True)

In [16]:
DELTA_RANGE = (0.5, 4)
THETA_RANGE = (5, 10)

In [17]:
def parallel_spectrogram_welch(sig, **kwargs):
    freqs, spg_times, spg = tfr.parallel_spectrogram_welch(sig.values, sig.fs, **kwargs)
    spg_times = spg_times + sig.time.values.min()
    return xr.DataArray(
        spg,
        dims=("frequency", "time", "channel"),
        coords={"frequency": freqs, "time": spg_times, "channel": sig.channel.values},
        attrs={'units': f"{sig.units}^2/Hz", 'file_start': sig.fileCreateTime}
    )

In [18]:
def get_bandpower(spg, f_range, chans):
    bandpower = spg.sel(frequency=slice(*f_range)).sum(
        dim="frequency"
    )
    bandpower.attrs["freq_range"] = f_range
    bandpower.attrs['chans'] = chans

    return bandpower

In [19]:
def get_file_bandpower(bin_path, bandpower_path, cx_chans, sr_chans):
    cx_sig = load_timeseries(bin_path, cx_chans)
    cx_spg = parallel_spectrogram_welch(cx_sig).median(dim="channel")
    cx_theta = get_bandpower(cx_spg, THETA_RANGE, cx_chans)
    cx_delta = get_bandpower(cx_spg, DELTA_RANGE, cx_chans)
    
    sr_sig = load_timeseries(bin_path, sr_chans)
    sr_spg = parallel_spectrogram_welch(sr_sig).median(dim="channel")
    sr_theta = get_bandpower(sr_spg, THETA_RANGE, sr_chans)
    sr_delta = get_bandpower(sr_spg, DELTA_RANGE, sr_chans)
    
    bandpower = xr.Dataset({'mpta_delta': cx_delta, 'mpta_theta': cx_theta, 'sr_delta': sr_delta, 'sr_theta': sr_theta})
    
    # Create parent directories if they do not already exist.
    Path(bandpower_path).parent.mkdir(parents=True, exist_ok=True)
    
    bandpower.to_netcdf(bandpower_path)
    bandpower.close()

In [20]:
def get_condition_bandpower(subject, condition):
    cx_chans = channel_groups.superficial_ctx[subject]
    sr_chans = channel_groups.stratum_radiatum_140um_to_200um[subject]
    
    bin_paths = paths.get_sglx_style_datapaths(subject=subject, experiment="sleep-homeostasis", condition=condition, ext="lf.bin")
    pow_paths = paths.get_sglx_style_datapaths(subject=subject, experiment="sleep-homeostasis", condition=condition, ext="pow.nc")

    for bin_path, pow_path in zip(bin_paths, pow_paths):
        get_file_bandpower(bin_path, pow_path, cx_chans, sr_chans)
        current_time = datetime.now().strftime("%H:%M:%S")
        print(f"{current_time}: Finished {str(bin_path)}")

# Run automated pipeline

In [28]:
get_condition_bandpower("Segundo", "dark-period-circadian-match")

nChan: 385, nFileSamp: 18000000
nChan: 385, nFileSamp: 18000000
15:58:58: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX2-Segundo/1-21-2020/1-21-2020_g0/1-21-2020_g0_imec0/1-21-2020_g0_t14.imec0.lf.bin
nChan: 385, nFileSamp: 18000000
nChan: 385, nFileSamp: 18000000
16:04:06: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX2-Segundo/1-21-2020/1-21-2020_g0/1-21-2020_g0_imec0/1-21-2020_g0_t15.imec0.lf.bin
nChan: 385, nFileSamp: 18000000
nChan: 385, nFileSamp: 18000000
16:09:39: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX2-Segundo/1-21-2020/1-21-2020_g0/1-21-2020_g0_imec0/1-21-2020_g0_t16.imec0.lf.bin
nChan: 385, nFileSamp: 18000000
nChan: 385, nFileSamp: 18000000
16:15:24: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX2-Segundo/1-21-2020/1-21-2020_g0/1-21-2020_g0_imec0/1-21-2020_g0_t17.imec0.lf.bin
nChan: 385, nFileSamp: 18000000
nChan: 385, nFileSamp: 18000000
16:20:24: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX2-Segundo/1-21-2020/1-21-20

In [29]:
get_condition_bandpower("Segundo", "dark-period")

nChan: 385, nFileSamp: 18000001
nChan: 385, nFileSamp: 18000001
16:33:10: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX2-Segundo/1-21-2020/1-21-2020_g0/1-21-2020_g0_imec0/1-21-2020_g0_t26.imec0.lf.bin
nChan: 385, nFileSamp: 18000001
nChan: 385, nFileSamp: 18000001
16:38:53: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX2-Segundo/1-21-2020/1-21-2020_g0/1-21-2020_g0_imec0/1-21-2020_g0_t27.imec0.lf.bin
nChan: 385, nFileSamp: 18000001
nChan: 385, nFileSamp: 18000001
16:44:32: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX2-Segundo/1-21-2020/1-21-2020_g0/1-21-2020_g0_imec0/1-21-2020_g0_t28.imec0.lf.bin
nChan: 385, nFileSamp: 18000001
nChan: 385, nFileSamp: 18000001
16:49:33: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX2-Segundo/1-21-2020/1-21-2020_g0/1-21-2020_g0_imec0/1-21-2020_g0_t29.imec0.lf.bin
nChan: 385, nFileSamp: 18000001
nChan: 385, nFileSamp: 18000001
16:54:20: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX2-Segundo/1-21-2020/1-21-20

In [30]:
get_condition_bandpower("Valentino", "dark-period-circadian-match")

nChan: 385, nFileSamp: 18000000
nChan: 385, nFileSamp: 18000000
17:04:23: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX3-Valentino/2-19-2020/2-19-2020_g1/2-19-2020_g1_imec0/2-19-2020_g1_t6.imec0.lf.bin
nChan: 385, nFileSamp: 18000000
nChan: 385, nFileSamp: 18000000
17:08:46: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX3-Valentino/2-19-2020/2-19-2020_g1/2-19-2020_g1_imec0/2-19-2020_g1_t7.imec0.lf.bin
nChan: 385, nFileSamp: 18000000
nChan: 385, nFileSamp: 18000000
17:13:30: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX3-Valentino/2-19-2020/2-19-2020_g1/2-19-2020_g1_imec0/2-19-2020_g1_t8.imec0.lf.bin
nChan: 385, nFileSamp: 18000000
nChan: 385, nFileSamp: 18000000
17:17:58: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX3-Valentino/2-19-2020/2-19-2020_g1/2-19-2020_g1_imec0/2-19-2020_g1_t9.imec0.lf.bin
nChan: 385, nFileSamp: 18000000
nChan: 385, nFileSamp: 18000000
17:23:24: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX3-Valentino/2-19-2020/2

In [31]:
get_condition_bandpower("Valentino", "dark-period")

nChan: 385, nFileSamp: 18000000
nChan: 385, nFileSamp: 18000000
17:32:58: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX3-Valentino/2-19-2020/2-20-2020_g0/2-20-2020_g0_imec0/2-20-2020_g0_t6.imec0.lf.bin
nChan: 385, nFileSamp: 18000000
nChan: 385, nFileSamp: 18000000
17:37:39: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX3-Valentino/2-19-2020/2-20-2020_g0/2-20-2020_g0_imec0/2-20-2020_g0_t7.imec0.lf.bin
nChan: 385, nFileSamp: 18000000
nChan: 385, nFileSamp: 18000000
17:42:25: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX3-Valentino/2-19-2020/2-20-2020_g0/2-20-2020_g0_imec0/2-20-2020_g0_t8.imec0.lf.bin
nChan: 385, nFileSamp: 18000000
nChan: 385, nFileSamp: 18000000
17:47:04: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX3-Valentino/2-19-2020/2-20-2020_g0/2-20-2020_g0_imec0/2-20-2020_g0_t9.imec0.lf.bin
nChan: 385, nFileSamp: 18000000
nChan: 385, nFileSamp: 18000000
17:52:17: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX3-Valentino/2-19-2020/2

In [32]:
get_condition_bandpower("Doppio", "dark-period-circadian-match")

nChan: 385, nFileSamp: 18000019
nChan: 385, nFileSamp: 18000019
18:01:34: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX4-Doppio/3-17-2020/3-17-2020_g0/3-17-2020_g0_imec0/3-17-2020_g0_t6.imec0.lf.bin
nChan: 385, nFileSamp: 18000019
nChan: 385, nFileSamp: 18000019


PermissionError: [Errno 13] Permission denied: b'/Volumes/neuropixel/Data/CNPIX4-Doppio/3-17-2020/3-17-2020_g0/3-17-2020_g0_imec0/3-17-2020_g0_t7.imec0.pow.nc'

In [ ]:
get_condition_bandpower("Doppio", "dark-period")

In [ ]:
get_condition_bandpower("Alessandro", "dark-period-circadian-match")

In [ ]:
get_condition_bandpower("Alessandro", "dark-period")

In [ ]:
get_condition_bandpower("Eugene", "dark-period-circadian-match")

In [ ]:
get_condition_bandpower("Eugene", "dark-period")

In [ ]:
get_condition_bandpower("Allan", "dark-period-circadian-match")

In [ ]:
get_condition_bandpower("Allan", "dark-period")

# Run pipeline piecemeal

In [11]:
%matplotlib widget

In [12]:
import matplotlib.pyplot as plt
from ecephys.plot import plot_spectrogram, plot_hypnogram_overlay
from ecephys.scoring import load_visbrain_hypnogram
from neurodsp.plts.time_series import plot_time_series
from ripple_detection.core import gaussian_smooth
smoothing_sigma = 2 # seconds

In [13]:
SUBJECT="Segundo"
CONDITION="REC-2-0"
START_TIME = 0
END_TIME = 60
bin_path = paths.get_datapath_from_csv(subject=SUBJECT, condition=CONDITION, data="lf.bin")

## Cortex

In [14]:
cx_chans = channel_groups.superficial_ctx[SUBJECT]
cx_sig = load_timeseries(bin_path, cx_chans, start_time=START_TIME, end_time=END_TIME, xarray=True)

nChan: 385, nFileSamp: 18000001


In [104]:
cx_spg = get_spectrogram(cx_sig)

In [105]:
cx_theta = _get_bandpower(cx_spg, THETA_RANGE, cx_chans)
cx_delta = _get_bandpower(cx_spg, DELTA_RANGE, cx_chans)

### Plot

In [36]:
plot_spectrogram(cx_spg.frequency, cx_spg.time, cx_spg, yscale='log')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [37]:
_, ax = plt.subplots(figsize=(24, 3))
plot_time_series(cx_spg.time.values, cx_delta.values, ax=ax, title="Cortical Delta", ylabel="Power")
#plot_hypnogram_overlay(hypno, ax=ax)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [38]:
_, ax = plt.subplots(figsize=(24, 3))
plot_time_series(cx_spg.time.values, cx_theta.values, ax=ax, title="Cortical Theta", ylabel="Power")
#plot_hypnogram_overlay(hypno, ax=ax)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [39]:
_, ax = plt.subplots(figsize=(24, 3))
plot_time_series(cx_spg.time.values, cx_theta.values / cx_delta.values, title="Cortical theta/delta ratio", ylabel="Ratio", ax=ax)
#plot_hypnogram_overlay(hypno, ax=ax)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Theta

In [106]:
sr_chans = channel_groups.stratum_radiatum_140um_to_200um[SUBJECT]
sr_sig = load_timeseries(bin_path, sr_chans, start_time=START_TIME, end_time=END_TIME, xarray=True)

nChan: 385, nFileSamp: 18000001


In [107]:
sr_spg = get_spectrogram(sr_sig)

In [108]:
sr_theta = _get_bandpower(sr_spg, THETA_RANGE, sr_chans)
sr_delta = _get_bandpower(sr_spg, DELTA_RANGE, sr_chans)

### Plot

In [43]:
plot_spectrogram(sr_spg.frequency, sr_spg.time, sr_spg, yscale='log')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [44]:
_, ax = plt.subplots(figsize=(20, 3))
plot_time_series(sr_spg.time.values, sr_delta.values, title="SR Delta", ylabel="Power", ax=ax)
#plot_hypnogram_overlay(hypno, ax=ax)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [45]:
_, ax = plt.subplots(figsize=(20, 3))
plot_time_series(sr_spg.time.values, sr_theta.values, title="SR Theta", ylabel="Power", ax=ax)
#plot_hypnogram_overlay(hypno, ax=ax)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [46]:
_, ax = plt.subplots(figsize=(20, 3))
plot_time_series(sr_spg.time.values, sr_theta.values / sr_delta.values, title="SR theta/delta ratio", ylabel="Ratio", ax=ax)
#plot_hypnogram_overlay(hypno, ax=ax)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [47]:
_, ax = plt.subplots(figsize=(20, 3))
plot_time_series(sr_spg.time.values, np.log10(sr_theta.values) / np.log10(sr_delta.values), title="SR log(theta)/log(delta) ratio", ylabel="Ratio", ax=ax)
#plot_hypnogram_overlay(hypno, ax=ax)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [48]:
_, ax = plt.subplots(figsize=(24, 3))
plot_time_series(sr_spg.time.values, gaussian_smooth(sr_theta.values / sr_delta.values, smoothing_sigma, 1 / np.diff(sr_spg.time.values).mean()), title="Smoothed SR theta/delta ratio", ylabel="Ratio", ax=ax)
#plot_hypnogram_overlay(hypno, ax=ax)

/home/gfindlay/miniconda3/envs/ecephys/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Save

In [109]:
bandpower = xr.Dataset({'mpta_delta': cx_delta, 'mpta_theta': cx_theta, 'sr_delta': sr_delta, 'sr_theta': sr_theta})

In [110]:
bandpower.to_netcdf(paths.get_datapath(subject=SUBJECT, condition=CONDITION, data="lfp_bandpower.nc"))

In [111]:
bandpower.close()